Project Summary
- Predicting future oil production rates for 5 oil wells
Data Import
Data Summary
- 5 wells in the same pool 
- Monthly data
- Variables:
    - Oil, gas, water production
    - WTI, Ed light, AECO pricing
EDA
- Time series plot
- Moving averages plot (6, 12, 36 months)
- Month to month percent change plot (df.pct_change())
- Trend-Seasonality-Residuals facet plot (TSR on y, wells on x)
- Autocorrelation/Lag facet plot (statsmodel, 5 wells on y, {6, 12, 36 months} on x)
- Pairplot production histograms
- Production correlation matrix
Data Cleaning & Preprocessing
- Add columns
    - Datetime (months since first production)
    - On/off production dummy
    - oil price dummy vars
- Normalization of production and pricing (MinMaxScaler(feature_range=(-1, 1)))
- Boxplots of production by year for all wells
- Train test split
    - Test is last 36 months, train is everything before that
    - 5 Bar plots with train vs test colored
Models
- LSTM
- Training loss plot
- y_true and y_pred scatter plot
- RMSE between y_true and y_pred
Hyperparameter Tuning
- Keras tuner
Other Models
- ARMIA (will need to make data stationary)
- Exponential Smoothing Time Series (ETS)
- Prophet

#### Examples
1. https://www.kaggle.com/code/faressayah/stock-market-analysis-prediction-using-lstm  
2. https://towardsdatascience.com/time-series-forecasting-a-practical-guide-to-exploratory-data-analysis-a101dc5f85b1/

#### Import Python Libraries

In [1]:
import pandas as pd
import requests
from fake_useragent import UserAgent
from io import StringIO
import json

#### Import Production Data

In [71]:
# Import production dataset
prod_df = pd.read_csv('prod.csv')

# Convert date column to datetime
prod_df['date'] = pd.to_datetime(prod_df['date'])

In [72]:
prod_df[['oil_volume (bbl)', 'gas_volume (mcf)', 'water_volume (bbl)']] = prod_df[['oil_volume (bbl)', 'gas_volume (mcf)', 'water_volume (bbl)']] * 100

In [73]:
prod_df['oil_producing_daily_avg (bbl/day)']  = prod_df['oil_volume (bbl)'] / prod_df['production_hours'] * 24
prod_df['gas_producing_daily_avg (mcf/day)']  = prod_df['gas_volume (mcf)'] / prod_df['production_hours'] * 24
prod_df['water_producing_daily_avg (bbl/day)']  = prod_df['water_volume (bbl)'] / prod_df['production_hours'] * 24

In [74]:
prod_df

,well_name,date,production_hours,oil_volume (bbl),gas_volume (mcf),water_volume (bbl),oil_producing_daily_avg (bbl/day),gas_producing_daily_avg (mcf/day),water_producing_daily_avg (bbl/day)
0,well_1,2013-01-01,408,279708.0,86167.0,24467.0,16453.411765,5068.647059,1439.235294
1,well_1,2013-02-01,629,423556.0,133842.0,10189.0,16161.119237,5106.848967,388.769475
2,well_1,2013-03-01,743,491737.0,300173.0,10378.0,15883.833109,9696.032301,335.224764
3,well_1,2013-04-01,720,377074.0,326659.0,3585.0,12569.133333,10888.633333,119.500000
4,well_1,2013-05-01,696,301911.0,220009.0,4906.0,10410.724138,7586.517241,169.172414
...,...,...,...,...,...,...,...,...,...
477,well_5,2018-08-01,120,503.0,353.0,126.0,100.600000,70.600000,25.200000
478,well_5,2018-09-01,720,1635.0,5650.0,7925.0,54.500000,188.333333,264.166667
479,well_5,2019-03-01,27,0.0,1059.0,0.0,0.000000,941.333333,0.000000
480,well_5,2019-04-01,24,0.0,1413.0,0.0,0.000000,1413.000000,0.000000


#### Import Pricing Data

In [3]:
# Import pricing dataset
oil_url = 'https://mcdan.com/forecast/data/oil.json'
gas_url = 'https://mcdan.com/forecast/data/gas.json'

def import_price_data(url):
    response = requests.get(url, headers={'User-Agent': UserAgent().random})

    data = json.loads(response.content)

    rows = []
    for entry in data:
        date = entry['date']
        row = {'date': date}
        for item in entry['values']:
            base_key = item['key'].replace(' Forecast', '').strip()
            row[f'{base_key}'] = item['usd']
        rows.append(row)
    df = pd.DataFrame(rows)
    df['date'] = pd.to_datetime(df['date'])
    return df

oil_prices_df = import_price_data(oil_url)
gas_prices_df = import_price_data(gas_url)

#### Merge Production & Pricing Datasets

In [17]:
df = oil_prices_df.merge(gas_prices_df).merge(prod_df)
df

,date,WTI,BRENT,EDM,WCS,HH,AECO,STA2,well_name,production_hours,oil_volume (bbl),gas_volume (mcf),water_volume (bbl),oil_producing_daily_avg (bbl/day),gas_producing_daily_avg (mcf/day),water_producing_daily_avg (bbl/day)
0,2011-04-01,110.04,123.33,115.59,89.65,4.24,3.78,3.46,well_4,150,3354.36,798.11,0.00,536.70,127.70,0.00
1,2011-05-01,101.36,114.99,104.22,85.60,4.31,3.82,3.41,well_4,252,2153.00,388.46,0.00,205.05,37.00,0.00
2,2011-06-01,96.29,113.83,99.94,79.40,4.54,4.31,3.73,well_4,672,3481.41,639.19,147.18,124.34,22.83,5.26
3,2011-07-01,97.31,116.97,99.70,78.31,4.42,4.21,3.90,well_4,744,3802.82,921.71,178.63,122.67,29.73,5.76
4,2011-08-01,86.34,110.22,89.65,66.45,4.05,3.70,3.37,well_4,125,411.98,60.03,72.33,79.10,11.53,13.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,2022-01-01,83.22,86.51,78.35,65.83,4.38,3.42,3.77,well_4,384,16.35,120.07,2.52,1.02,7.50,0.16
478,2022-05-01,109.55,113.34,108.70,96.83,8.14,4.85,4.80,well_2,1,0.00,3.53,0.00,0.00,84.75,0.00
479,2023-01-01,78.12,82.50,75.03,49.89,3.27,4.54,4.46,well_4,1,0.00,3.53,0.00,0.00,84.75,0.00
480,2023-05-01,71.58,75.47,67.34,56.18,2.15,1.59,0.91,well_2,24,0.00,3.53,0.00,0.00,3.53,0.00


#### Data Summary

In [55]:
df.shape

(482, 17)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 482 entries, 0 to 481
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   date                                 482 non-null    datetime64[ns]
 1   WTI                                  482 non-null    float64       
 2   BRENT                                482 non-null    float64       
 3   EDM                                  482 non-null    float64       
 4   WCS                                  482 non-null    float64       
 5   HH                                   482 non-null    float64       
 6   AECO                                 482 non-null    float64       
 7   STA2                                 482 non-null    float64       
 8   well_name                            482 non-null    object        
 9   production_hours                     482 non-null    int64         
 10  oil_volume (bb

## EDA

## Data Cleaning & Preprocessing

#### Add Columns

In [52]:
# Add a months_on column which is the number of months a well has produced since it was first put on production
df['months_on'] = df.groupby('well_name')['date'].transform(lambda s: s.rank(method='first').astype(int))
df

,date,WTI,BRENT,EDM,WCS,HH,AECO,STA2,well_name,production_hours,oil_volume (bbl),gas_volume (mcf),water_volume (bbl),oil_producing_daily_avg (bbl/day),gas_producing_daily_avg (mcf/day),water_producing_daily_avg (bbl/day),months_on
0,2011-04-01,110.04,123.33,115.59,89.65,4.24,3.78,3.46,well_4,150,3354.36,798.11,0.00,536.70,127.70,0.00,1
1,2011-05-01,101.36,114.99,104.22,85.60,4.31,3.82,3.41,well_4,252,2153.00,388.46,0.00,205.05,37.00,0.00,2
2,2011-06-01,96.29,113.83,99.94,79.40,4.54,4.31,3.73,well_4,672,3481.41,639.19,147.18,124.34,22.83,5.26,3
3,2011-07-01,97.31,116.97,99.70,78.31,4.42,4.21,3.90,well_4,744,3802.82,921.71,178.63,122.67,29.73,5.76,4
4,2011-08-01,86.34,110.22,89.65,66.45,4.05,3.70,3.37,well_4,125,411.98,60.03,72.33,79.10,11.53,13.89,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,2022-01-01,83.22,86.51,78.35,65.83,4.38,3.42,3.77,well_4,384,16.35,120.07,2.52,1.02,7.50,0.16,126
478,2022-05-01,109.55,113.34,108.70,96.83,8.14,4.85,4.80,well_2,1,0.00,3.53,0.00,0.00,84.75,0.00,115
479,2023-01-01,78.12,82.50,75.03,49.89,3.27,4.54,4.46,well_4,1,0.00,3.53,0.00,0.00,84.75,0.00,127
480,2023-05-01,71.58,75.47,67.34,56.18,2.15,1.59,0.91,well_2,24,0.00,3.53,0.00,0.00,3.53,0.00,116


#### Create Individual Well Datasets

In [54]:
well_1_df = df[df['well_name']=='well_1']
well_2_df = df[df['well_name']=='well_2']
well_3_df = df[df['well_name']=='well_3']
well_4_df = df[df['well_name']=='well_4']
well_5_df = df[df['well_name']=='well_5']